In [1]:
!pip install -qq transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 938.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.7 MB/s eta 0:00:00


In [18]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import create_optimizer
from transformers import DefaultDataCollator
from transformers import TFAutoModelForQuestionAnswering

import tensorflow as tf
import numpy as np

In [3]:
train_sberquad = load_dataset("sberquad", split="train")
valid_sberquad = load_dataset("sberquad", split="validation")
test_sberquad = load_dataset("sberquad", split="test")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

Dataset sberquad downloaded and prepared to /root/.cache/huggingface/datasets/sberquad/sberquad/1.0.0/62115d937acf2634cfacbfee10c13a7ee39df3ce345bb45af7088676f9811e77. Subsequent calls will reuse this data.


In [4]:
tokenizer = AutoTokenizer.from_pretrained("Geotrend/distilbert-base-ru-cased")

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/133k [00:00<?, ?B/s]

In [5]:
train_sberquad[0]

{'id': 62310,
 'title': 'SberChallenge',
 'context': 'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.',
 'question': 'чем представлены органические остатки?',
 'answers': {'text': ['известковыми выделениями сине-зелёных водорослей'],
  'answer_start': [109]}}

In [6]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [7]:
tokenized_train_sberquad = train_sberquad.map(preprocess_function, batched=True)
tokenized_valid_sberquad = valid_sberquad.map(preprocess_function, batched=True)
tokenized_test_sberquad = test_sberquad.map(preprocess_function, batched=True)

  0%|          | 0/46 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/24 [00:00<?, ?ba/s]

In [8]:
data_collator = DefaultDataCollator(return_tensors="tf")

In [9]:
batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_train_sberquad) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [10]:
model = TFAutoModelForQuestionAnswering.from_pretrained("Geotrend/distilbert-base-ru-cased", from_pt=True)

Downloading:   0%|          | 0.00/218M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream

In [11]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_train_sberquad,
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_valid_sberquad,
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [12]:
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [13]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3)

Epoch 1/3
2833/2833 [==============================] - 1982s 697ms/step - loss: 2.4559 - val_loss: 1.9436
Epoch 2/3
2833/2833 [==============================] - 1991s 703ms/step - loss: 1.8545 - val_loss: 1.8451
Epoch 3/3
2833/2833 [==============================] - 1990s 702ms/step - loss: 1.7118 - val_loss: 1.8451


In [14]:
inputs = tokenizer(test_sberquad[0]["question"], test_sberquad[0]["context"], return_tensors="tf")

In [15]:
test_sberquad[0]["question"], test_sberquad[0]["context"]

('У каких организмов отсутствуют настоящие дифференцированные клетки?',
 'Многоклеточный организм — внесистематическая категория живых организмов, тело которых состоит из многих клеток, большая часть которых (кроме стволовых, например, клеток камбия у растений) дифференцированы, то есть различаются по строению и выполняемым функциям. Следует отличать многоклеточность и колониальность. У колониальных организмов отсутствуют настоящие дифференцированные клетки, а следовательно, и разделение тела на ткани. Граница между многоклеточностью и колониальностью нечёткая. Например, вольвокс часто относят к колониальным организмам, хотя в его колониях есть чёткое деление клеток на генеративные и соматические. Кроме дифференциации клеток, для многоклеточных характерен и более высокий уровень интеграции, чем для колониальных форм. Многоклеточные животные, возможно, появились на Земле 2,1 миллиарда лет назад, вскоре после кислородной революции .')

In [16]:
output = model(**inputs)

answer_start_index = int(tf.math.argmax(output.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(output.end_logits, axis=-1)[0])

In [17]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'У колониальных'

In [24]:
random_indices = np.random.randint(0, len(test_sberquad), size=10)

for i, idx in enumerate(random_indices, 1):
    idx = idx.item()
    print(f"Question {i}: {test_sberquad[idx]['question']}")

    inputs = tokenizer(test_sberquad[idx]["question"], test_sberquad[idx]["context"], return_tensors="tf")
    output = model(**inputs)

    answer_start_index = int(tf.math.argmax(output.start_logits, axis=-1)[0])
    answer_end_index = int(tf.math.argmax(output.end_logits, axis=-1)[0])

    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    answer = tokenizer.decode(predict_answer_tokens)

    print(f"The answer is: {answer}")

Question 1: В каком году Полибий был избран на пост союзного гиппарха?
The answer is: В 169 до н. э.
Question 2: Во время работы на чем Николай Карамзин обращался к Тациту?
The answer is: [SEP] К Тациту обращался Николай Карамзин во время работы над Историей государства российского
Question 3: В каком году был основан Плантаторский банк Ямайки?
The answer is: В мае 1836
Question 4: Из чего образуются зародыши?
The answer is: из неоплодотворённых диплоидных яйцеклеток или клеток тапетума
Question 5: Как звали советского кинематографиста Пудовкина?
The answer is: Всеволод
Question 6: Какой наиболее популярный памятник испанского барокко?
The answer is: Собор Святого Иакова
Question 7: Какие виды рыб встречаются в акваториях города?
The answer is: ##бров. В акваториях города встречаются виды рыб, характерные в целом для этой части Финского залива : щука, окунь, судак, лещ, плотва, краснопёрка и другие
Question 8: Какие процессы человек изучил вместе с освоением огня?
The answer is: горени